In [1]:
!pip install -q pyspellchecker

In [2]:
import pandas as pd
import numpy as np
import re 
import string 
import warnings
from spellchecker import SpellChecker

warnings.filterwarnings("ignore")


import torch
import os

from PIL import Image

import json
import matplotlib.pyplot as plt

import pandas as pd
import torch
import torch.nn as nn
from tqdm import tqdm
from time import sleep
import warnings
from transformers import AdamW
import spacy

from transformers import CLIPProcessor, CLIPModel
import torch.nn.functional as F

device = device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

warnings.filterwarnings("ignore")

nlp = spacy.load("en_core_web_sm")

In [3]:
# read annotations file

Task1_Train = pd.read_csv("/kaggle/input/subtask-b/individual_stB_train.csv")
Task1_Dev = pd.read_csv("/kaggle/input/subtask-b/stB_eval.csv")
Task1_Test = pd.read_csv("/kaggle/input/subtask-b/stB_test.csv")

Task1_Train = Task1_Train.reset_index()

Task1_Train['ORG'] = 0
Task1_Train['PER'] = 0
Task1_Train['NOORG'] = 0

Task1_Test['ORG'] = 0
Task1_Test['PER'] = 0
Task1_Test['NOORG'] = 0

Task1_Train.dropna(inplace=True)

In [4]:
def remove_emojis(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # Emoticons
                               u"\U0001F300-\U0001F5FF"  # Symbols & Pictographs
                               u"\U0001F680-\U0001F6FF"  # Transport & Map Symbols
                               u"\U0001F700-\U0001F77F"  # Alphanumeric Supplement
                               u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
                               u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
                               u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
                               u"\U0001FA00-\U0001FA6F"  # Chess Symbols
                               u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
                               u"\U00002702-\U000027B0"  # Dingbats
                               u"\U000024C2-\U0001F251" 
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def remove_punctuation(input_string):
    translator = str.maketrans('', '', string.punctuation)
    
    result = input_string.translate(translator)
    
    return result

def remove_non_english_chars(input_string):
    translation_table = dict.fromkeys(
        i for i in range(sys.maxunicode) if unicodedata.category(chr(i)).startswith('C')
    )

    result = input_string.translate(translation_table)

    return result

def correct_spelling(input_text):
    try:
        # Create a SpellChecker object
        spell = SpellChecker()

        # Split the input text into words
        words = input_text.split()

        # Find misspelled words
        misspelled = spell.unknown(words)

        # Correct misspelled words
        corrected_text = " ".join(spell.correction(word) if word in misspelled else word for word in words)

        return corrected_text

    except Exception as e:
        print(f"An error occurred: {e}")
        return input_text
    
hyperlink_pattern = r'https?://\S+|www\.\S+'

def PreProcessTweets(tweet):
  tweet2 = str(tweet)
  tweet2 = re.sub(r'^RT[\s]+', '', tweet)
  tweet2 = re.sub(r'#([^\s]+)', '', tweet2)
  tweet2 = tweet2.replace("LINK","")
  tweet2 = tweet2.replace("&amp","") 
  tweet2 = re.sub(r'@', '', tweet2)
  tweet2 = tweet2.replace("\n","")
  tweet2 = re.sub(r'_', ' ', tweet2)
  tweet2 = tweet2.replace("UN","United Nations ")
  tweet2 = re.sub(hyperlink_pattern, '', tweet2)
  tweet2 = tweet2.replace(":","") 
  tweet2 = re.sub('‼‼‼','', tweet2)
  tweet2 = remove_emojis(tweet2)
  tweet2 = remove_punctuation(tweet2)
  tweet2 = re.sub(' +', ' ',tweet2)
  tweet2 = tweet2.replace("youve","you have")
  tweet2 = tweet2.replace("govt","Government")
  #tweet2 = correct_spelling(tweet2)
  return tweet2

Task1_Train['text'] = Task1_Train['text'].apply(lambda x: PreProcessTweets(x))
Task1_Dev['text'] = Task1_Dev['text'].apply(lambda x: PreProcessTweets(x))
Task1_Test['text'] = Task1_Test['text'].apply(lambda x: PreProcessTweets(x))

In [5]:
ImagePathTest = '/kaggle/input/subtask-b/TestData/subtaskB/'

for i in range(len(Task1_Test)):
    Task1_Test['filename'][i] = ImagePathTest + Task1_Test['filename'][i].split('/')[-1]

In [6]:
Indvidual = '/kaggle/input/subtask-b/subTaskB-20231121T151628Z-001/subTaskB/Individual/'
Community = '/kaggle/input/subtask-b/subTaskB-20231121T151628Z-001/subTaskB/Community/'
Organization = '/kaggle/input/subtask-b/subTaskB-20231121T151628Z-001/subTaskB/Organization/'

# HatePath = '/kaggle/input/task-a/subTaskA-20231121T171317Z-001/subTaskA/Hate Speech/'
# NonHatePath = '/kaggle/input/task-a/subTaskA-20231121T171317Z-001/subTaskA/No Hate Speech/'

for i in range(len(Task1_Train)):
    if(Task1_Train['label'][i] == 0):
        Task1_Train['filename'][i] = Indvidual + Task1_Train['filename'][i].split('/')[-1]
    elif(Task1_Train['label'][i] == 1):
        Task1_Train['filename'][i] = Community + Task1_Train['filename'][i].split('/')[-1]
    else:
        Task1_Train['filename'][i] = Organization + Task1_Train['filename'][i].split('/')[-1]
#     elif(Task1_Train['label'][i] == 2):
#         Task1_Train['filename'][i] = Organization + Task1_Train['filename'][i].split('/')[-1]
 
# DevPath = '/kaggle/input/subtask-b/subtaskB-20231121T151609Z-001/subtaskB/'

# for i in range(len(Task1_Dev)):
#     if(Task1_Dev['label'][i] == 0):
#         Task1_Dev['filename'][i] = DevPath + Task1_Dev['filename'][i].split('/')[-1]
#     else:
#         Task1_Dev['filename'][i] = DevPath + Task1_Dev['filename'][i].split('/')[-1]

In [8]:
def NERFeatureExtractor(text, nlp = nlp):
    doc = nlp(text)
    PersonCount = NORPCount = ORGCount = 0
    for ent in doc.ents:
        if ent.label_ == 'PERSON':
            PersonCount = 1
        elif ent.label_ == 'NORP':
            NORPCount = 1
        elif ent.label_ == 'ORG':
            ORGCount = 1
    NERFeaturesReady = {'PersonCount' : PersonCount, 'NORPCount' : NORPCount , 'ORGCount' : ORGCount}
    return NERFeaturesReady

In [65]:
%%time
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("summarization", model="currentlyexhausted/flan-t5-summarizer", device = "cuda:0", max_length = 70) 

for i in range(len(Task1_Train)):
    if(len(Task1_Train['text'][i].split()) > 100):
        Task1_Train['text'][i] = pipe(Task1_Train['text'][i])[0]['summary_text']

for i in range(len(Task1_Test)):
    if(len(Task1_Test['text'][i].split()) > 90):
        Task1_Test['text'][i] = pipe(Task1_Test['text'][i])[0]['summary_text']

Token indices sequence length is longer than the specified maximum sequence length for this model (1189 > 512). Running this sequence through the model will result in indexing errors


CPU times: user 14 s, sys: 191 ms, total: 14.2 s
Wall time: 14.3 s


In [46]:
class MultiModalDataset(torch.utils.data.Dataset):
    def __init__(self, images, texts, labels, image_transform=None, text_tokenizer=None):
        self.images = images
        self.texts = texts
        self.labels = labels
        self.image_transform = image_transform
        self.text_tokenizer = text_tokenizer

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = self.images[idx]
        text = self.texts[idx]
        label = self.labels[idx]
        
        image = Image.open(image)
        
        sample = {}
        # Image preprocessing
        if self.image_transform:
            image = self.image_transform(image)

        # Text preprocessing
        if self.text_tokenizer:
            text = self.text_tokenizer(text, return_tensors="pt")['input_ids'].squeeze()

        try:
            sample["label"] = int(label)
            sample["text"] = text
            sample["image"] = image
        except Exception as e:
            print(e)
        
        return sample

In [56]:
IMG_SIZE = 224

image_train = transforms.Compose(
    [
        transforms.ToTensor(),
    ]
)

Multimodaldataset = MultiModalDataset(Task1_Train['filename'].values, Task1_Train['text'].values, Task1_Train['label'].values, image_transform=image_train)

Multimodaltestdataset = MultiModalDataset(Task1_Test['filename'].values, Task1_Test['text'].values, Task1_Test['label'].values, image_transform=image_train)

In [64]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from transformers import RobertaModel, RobertaTokenizer

class MultimodalClassifier(nn.Module):
    def __init__(self, vision_model, text_model, num_classes):
        super(MultimodalClassifier, self).__init__()
        self.vision_model = vision_model
        self.text_model = text_model
        self.fc = nn.Sequential(
            nn.Linear(1000 + text_model.config.hidden_size, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, images, inputs):
        
        # Image processing with ViT
        vision_output = self.vision_model(**images)
        vision_embedding = vision_output.logits

        # Text processing with RoBERTa
        text_output = self.text_model(**inputs)
        text_embedding = text_output.last_hidden_state[:, 0, :]

        # Concatenate image and text embeddings
        multimodal_embedding = torch.cat((vision_embedding, text_embedding), dim=1)

        # Classification
        linear_output = self.fc(multimodal_embedding)
        
        return F.log_softmax(linear_output, dim=1)

In [67]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from transformers import ViTFeatureExtractor, ViTForImageClassification
from transformers import AutoModel, AutoTokenizer


# Load pre-trained ViT and RoBERTa models
vit_model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')
vit_feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
roberta_model = AutoModel.from_pretrained('GroNLP/HateBERT')
tokenizer = AutoTokenizer.from_pretrained('GroNLP/HateBERT')

# Create the multimodal classifier
num_classes = 3  # Set the number of classes according to your task
multimodal_classifier = MultimodalClassifier(vit_model, roberta_model, num_classes)
multimodal_classifier = multimodal_classifier.to(device)

In [9]:
## Specify the Hyper parameters 

BATCH_SIZE = 4
NUM_LABELS = 3
EPOCHS = 10
LEARNING_RATE = 1e-5

torch.backends.cudnn.benchmark = True
torch.backends.cudnn.enabled   = True

In [10]:
# split into training, validation and testing

train_dataset, validation_dataset = torch.utils.data.random_split(dataset, [1642, 300])

In [11]:
## We call the dataloader class
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    pin_memory=True,
    num_workers=4,
    shuffle=True,
    drop_last=True
 )

test_loader = torch.utils.data.DataLoader(
    testdataset,
    batch_size=1,
    pin_memory=True,
    num_workers=0,
 )

val_loader = torch.utils.data.DataLoader(
    validation_dataset,
    batch_size=BATCH_SIZE,
    pin_memory=True,
    num_workers=4,
    shuffle=True,
    drop_last=True
 )

dataloaders = {'Train': train_loader,'Test': test_loader, 'Val': val_loader}

In [12]:
class Model(nn.Module):

    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(1792, 128)
        self.fc2 = nn.Linear(128, 3)
        self.dropout = nn.Dropout(0.25)

    def forward(self, input):
      linear_out = self.dropout(F.relu(self.fc1(input)))    
      final_out = self.fc2(linear_out)    
      return final_out

In [ ]:
import torch
from tqdm import tqdm
from PIL import Image

# Assuming you have already defined EPOCHS, model, dataloaders, criterion, optimizer, and device

train_loss = []
train_acc = []

val_loss = []
val_acc = []

for epoch in range(0, EPOCHS):
    print('-' * 50)
    print('Epoch {}/{}'.format(epoch + 1, EPOCHS))

    for phase in ['Train', 'Val']:
        batch_loss = 0.0  # live loss
        batch_acc = 0.0  # live accuracy

        y_pred = []
        y_true = []

        if phase == 'Train':
            multimodal_classifier.train()
        else:
            multimodal_classifier.eval()

        with tqdm(dataloaders[phase], unit="batch", desc=phase) as tepoch:

            for idx, batch in enumerate(tepoch):
                labels = batch["label"].to(device)
                text = batch["text"]

                imgs = []

                img_paths = batch["image"]
                
                image = vit_feature_extractor(images = img_paths, return_tensors="pt")
                ImageInputs = {key: value.to(device) for key, value in image.items()}

                inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
                inputs = {key: value.to(device) for key, value in inputs.items()}
                
                output = multimodal_classifier(ImageInputs, inputs)

                # Ensure the model parameters are on the same device as the input data
                output = output.to(device)
                

                loss = criterion(output, labels)

                if phase == 'Train':
                    # zero gradients
                    optimizer.zero_grad()

                    # Backward pass  (calculates the gradients)
                    loss.backward()

                    optimizer.step()  # Updates the weights

                batch_loss += loss.item()

                _, preds = output.data.max(1)

                y_pred.extend(preds.tolist())
                y_true.extend(labels.tolist())
                
                tepoch.set_postfix(loss=batch_loss / (idx + 1), accuracy=sum((y_pred[i] == y_true[i]) for i in range(len(y_true))) / len(y_true))

In [ ]:
y_pred = []
files = []

i = 0

with tqdm(dataloaders['Test'], unit="batch", desc=phase) as tepoch:
  for idx, batch in enumerate(tepoch):
    labels = batch["label"].to(device)
    text = batch["text"]
    imgs = []

    img_paths = batch["image"]

    for path in img_paths:
      i += 1
      files.append((path.split('/')[-1][:5]))
      imgs.append((Image.open(path)))
    
    image = vit_feature_extractor(images = img_paths, return_tensors="pt")
    ImageInputs = {key: value.to(device) for key, value in image.items()}

    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    inputs = {key: value.to(device) for key, value in inputs.items()}
                
    output = multimodal_classifier(ImageInputs, inputs)

    _, preds = output.data.max(1)

    y_pred.extend(preds.tolist())

In [23]:
import json

with open('submission.json', 'w') as json_file:
    for key, value in sorted_dict.items():
        predictionsDictionary = {'index': int(key), 'prediction': int(value)}
        json.dump(predictionsDictionary, json_file)    
        json_file.write('\n')    

In [145]:
import json

with open('submission.json', 'w') as json_file:
    for key, value in sorted_dict.items():
        predictionsDictionary = {'index': int(key), 'prediction': int(value)}
        json.dump(predictionsDictionary, json_file)    
        json_file.write('\n')    

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report, accuracy_score

In [ ]:
accuracy_score(y_true,y_pred)

In [ ]:
f1_score(y_true, y_pred, average='macro')

In [ ]:
precision_score(y_true, y_pred, average='macro')

In [ ]:
recall_score(y_true, y_pred, average='macro')

In [ ]:
from imblearn.metrics import macro_averaged_mean_absolute_error 
macro_averaged_mean_absolute_error(y_true, y_pred)